## 寫爬蟲抓蘋果新聞

In [5]:
import requests
from bs4 import BeautifulSoup
domain = 'http://www.appledaily.com.tw'
url_ary = []
for i in range(1,40):
    res = requests.get('http://www.appledaily.com.tw/realtimenews/section/new/{}'.format(i))
    soup = BeautifulSoup(res.text, 'html.parser')
    for a in soup.select('.rtddt a'):
        url_ary.append(domain + a['href'])

In [10]:
#print(url_ary)

In [12]:
articles = []
#url = 'http://www.appledaily.com.tw/realtimenews/article/international/20170616/1141992/%E6%B8%AF%E8%AD%B0%E5%93%A1%E8%A2%AB%E6%8C%87%E6%94%B6%E5%8F%97%E9%BB%8E%E6%99%BA%E8%8B%B1%E6%8D%90%E6%AC%BE%E6%A1%88%E3%80%80%E4%B8%8B%E6%9C%8831%E6%97%A5%E8%A3%81%E6%B1%BA'
for url in url_ary:
    try:
        dic = {}
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        dic['title'] = soup.select('#h1')[0].text
        dic['summary']=soup.select('#summary')[0].text
        dic['url'] = url
        articles.append(dic)
    except:
        pass

In [18]:
import pandas
newsdf = pandas.DataFrame(articles)
newsdf = newdf.drop_duplicates()
newsdf.to_excel('news.xlsx')

## 讀取新聞資料

In [21]:
import pandas
newsdf = pandas.read_excel('news.xlsx')

In [22]:

newsdf.head()

,summary,title,url
0,（更新：影片）連日豪大雨，連續多次民調倒數第一的嘉義市長涂醒哲，罕見地透過區政系統問各里里長...,吐氣揚眉「這次沒淹」 涂醒哲公開信宣揚,http://www.appledaily.com.tw/realtimenews/arti...
1,（更新戰況）15:288局下，2出局二、三壘有人，陽岱鋼未能把握機會，擊出三壘滾地球出局。雙...,【更新】陽岱鋼第4打席未能建功 雙方3比3平手,http://www.appledaily.com.tw/realtimenews/arti...
2,（更新：新增影片）高雄市黃姓男子（55歲，恐嚇、毀損前科）在傳統市場賣魚多年，覺得賣魚賺不了...,「蚵仔」當暗號 魚販賣毒被逮還比Ya！,http://www.appledaily.com.tw/realtimenews/arti...
3,(更新：男單決賽)15:11男子世界排名前2中國2名好手球王馬龍、20歲新星樊振東重演世界錦...,【更新】日本桌球公開賽 17歲中國新星首個國際賽就封后,http://www.appledaily.com.tw/realtimenews/arti...
4,法國精品品牌LOUIS VUITTON（路易威登，簡稱LV）的商品以卓越品質著稱，但英國《衛...,LV「義大利製造」名鞋 竟是羅馬尼亞生產的,http://www.appledaily.com.tw/realtimenews/arti...


## 資料預處理 (文章切詞, 轉詞頻矩陣)

In [24]:
import jieba

title_ary = []
article_ary = []

for news in newsdf.iterrows():
    article_ary.append(' '.join(jieba.cut(news[1].summary)))
    title_ary.append(news[1].title)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\NTCB\AppData\Local\Temp\jieba.cache
Loading model cost 1.123 seconds.
Prefix dict has been built succesfully.


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(article_ary)

## 計算 Cosine Similarity

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(X,X)

## 資料分群

In [46]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 30, init = 'k-means++', random_state = 123)
y_kmeans = kmeans.fit_predict(cosine_similarities)

In [61]:
import numpy 
from collections import Counter
#title_npary = numpy.array(title_ary)
Counter(y_kmeans)
for ele in title_npary[y_kmeans == 28]:
    print(ele)

​對陳水扁要以兩公約精神依病治理
扁醫師轟中監侵犯醫療人權　法務部：支持中監
【怒轟片】扁醫陳順勝：中監哪有權利撤換醫師
扁再出擊！　點名邱太三：敢說沒有政治力介入
扁斥離開住所要申請形同軟禁　中監說話了
【親筆信曝光】陳水扁怒批蔡政府：逼我死諫


In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
sil = []
for i in range(2, 30):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    y_kmeans = kmeans.fit_predict(X)
    sil.append(metrics.silhouette_score(X, y_kmeans))


In [ ]:
plt.plot(range(2, 30), sil)
plt.xlim([0,])
plt.title('The Silhouette Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Coefficient')
plt.show()

## 文字探勘

In [ ]:
a = 'this is a book'
a.split()

In [ ]:
b = '今天天氣真好' 

In [ ]:
! pip install jieba

In [ ]:
import jieba
for ele in jieba.cut(b):
    print(ele)

In [ ]:
jieba.add_word('何壽川')

In [ ]:
sentence = ['何壽川遭聲押　邱正雄緊急代理永豐金董事長','何壽川涉弊案　永豐金、金管會下周一嚴陣以待']
seg_ary =[]
for s in sentence:
    seg_ary.append(' '.join([w for w in jieba.cut(s)]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(seg_ary)

In [ ]:
print(vectorizer.get_feature_names())
X.toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(X,X)

In [ ]:
cosine_similarities